In [84]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
from scipy import ndimage
from numpy import unique
import copy as cp
from scipy.spatial import distance
from pyefd import elliptic_fourier_descriptors
import imageio

PATH_TO_PROJECT = "/Users/Desktop/COMP9517/Group/Sequences"
PATH_TO_IMAGES = PATH_TO_PROJECT + "/01"
PATH_TO_WATERSHED = PATH_TO_PROJECT + "/watershed"
PATH_TO_ENHANCED = PATH_TO_PROJECT + "/enhanced"
PATH_TO_SEEDS = PATH_TO_PROJECT + "/seeds"
PATH_TO_RESULTS = PATH_TO_PROJECT + "/results"

### Read Images

In [85]:
# Help functions for preprocessing images and improving image quality
def normalize(img):
    # Normalize a given image
    img_o = cv2.normalize(img, img, 0, 255, cv2.NORM_MINMAX)
    return img_o

def stretch(img):
    # Perform contrast stretching on the given image
    a, b = 0, 255
    c, d = np.min(img), np.max(img)
    img_o = (img - c) * ((b - a) / (d - c)) + a
    return img_o.astype('uint8')

def cvt_binary(img, threshold):
    # Perform Gaussian thresholding on a given image
    img = cv2.GaussianBlur(img,(5,5),0).astype(np.uint8)
    _, thresh = cv2.threshold(img,threshold,1,cv2.THRESH_BINARY)

    # Using morphological operations to imporve the quality of result
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(9,9))
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    
    return thresh

def write_images(images, name):
    # Write image as 8 bits image into folder with the given name
    for i, img in enumerate(images):
        if i < 10:
            cv2.imwrite(name+"00"+str(i)+".tif", img.astype(np.uint8))
        elif i >= 10 and i < 100:
            cv2.imwrite(name+"0"+str(i)+".tif", img.astype(np.uint8))
        else:
            cv2.imwrite(name+str(i)+".tif", img.astype(np.uint8))


In [86]:
path = PATH_TO_IMAGES
imgs_list = []
imgs_enhanced = []
imgs_normalized = []
imgs_stretched = []
for r,d,f in os.walk(path):
    f = sorted(f)
    i = 0
    for files in f:
        if files[-3:].lower()=='tif':
            temp = cv2.imread(os.path.join(r,files))
            gray = cv2.cvtColor(temp, cv2.COLOR_BGR2GRAY) 
            imgs_list.append(gray.copy())
            normalized = normalize(gray.copy())
            imgs_normalized.append(normalized)
            stretched = stretch(normalized.copy())
            imgs_stretched.append(stretched)
            binary = cvt_binary(stretched, 33)
            imgs_enhanced.append(binary)
            print ("Reading and enhancing image: ", i)
            i += 1
            clear_output(wait=True)
                  
print ("Complete!")

# Save images
for i,img in enumerate(imgs_enhanced):
    imgs_enhanced[i] = img*255

os.chdir(PATH_TO_ENHANCED)
write_images(imgs_enhanced, "enhanced")


Complete!


### Segmentation

In [87]:
# Find seed points (center) of cells to perform watershed
def distancemap(images):
    '''
    Generate a distance map image for the given images
    '''
    return [cv2.distanceTransform(images[i], distanceType=2, maskSize=0)\
            for i in range(len(images))]

def combine_images(images, alpha, dismap):
    '''
    Generate a new image combining the oringal image with
    the distance map image
    '''
    return [images[i] + alpha * dismap[i] for i in range(len(images))]

In [88]:
dismap = distancemap(imgs_enhanced)
imgs_combined = combine_images(imgs_list, 0.4, dismap) # take alpha as 0.4

out = []
pair = []
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
for i,img in enumerate(dismap):
    neighborhood_size = 20
    data_max = ndimage.filters.maximum_filter(img, neighborhood_size)
    data_max[data_max==0] = 255
    pair.append((img == data_max).astype(np.uint8))

    dilation = cv2.dilate((pair[i]*255).astype(np.uint8),kernel,iterations = 1)
    out.append(dilation)
    os.chdir(PATH_TO_SEEDS)
    print ("Finding seed points of image: ", i)
    write_images(dilation, "seed")
    clear_output(wait=True)
    
print ("Complete!")


Complete!


In [89]:
class WATERSHED():
    '''
    This class contains all the function to compute watershed.
    Reference: https://medium.com/@hh595/implement-cell-tracking-step-by-step-782466b34c59
    '''
    def __init__(self, images, markers):
        self.images = images
        self.markers = markers

    def watershed_compute(self):
        '''
        This function is to compute watershed given the newimage and the seed image. 
        '''
        result = []
        outmark = []
        outbinary = []

        for i in range(len(self.images)):
            print ("Performing watershed on image: image ", i)
            # generate a 3-channel image in order to use cv2.watershed
            imgcolor = np.zeros((self.images[i].shape[0], self.images[i].shape[1], 3), np.uint8)
            for c in range(3): 
                imgcolor[:,:,c] = self.images[i]

            # compute marker image (labelling)
            if len(self.markers[i].shape) == 3:
                self.markers[i] = cv2.cvtColor(self.markers[i],cv2.COLOR_BGR2GRAY)
            _, mark = cv2.connectedComponents(self.markers[i])
            
            # watershed
            mark = cv2.watershed(imgcolor,mark)
            
            u, counts = unique(mark, return_counts=True)
            counter = dict(zip(u, counts))
            for index in counter:
                temp_img = np.zeros_like(mark)
                temp_img[mark==index] = 255
                if counter[index] > 3000:
                    mark[mark==index] = 0
                    continue
            
            labels = list(set(mark[mark>0]))
            length = len(labels)
            temp_img = mark.copy()
            for original, new in zip(labels, range(1,length+1)):
                temp_img[mark==original] = new
            mark = temp_img
            
            # mark image and add to the result 
            temp = cv2.cvtColor(imgcolor,cv2.COLOR_BGR2GRAY)
            result.append(temp)
            outmark.append(mark.astype(np.uint8))

            binary = mark.copy()
            binary[mark>0] = 255
            outbinary.append(binary.astype(np.uint8))
            clear_output(wait=True)

        return result, outbinary, outmark

In [90]:
# watershed
ws = WATERSHED(imgs_combined, pair) 
wsimage, bmarks, marks = ws.watershed_compute()

os.chdir(PATH_TO_WATERSHED)
write_images(bmarks, "watershed")
print ("Complete!")

Complete!


### Tracking
Find the similarity of cells using the combination of centroid distances and shapes.

In [91]:
class CELL():
    '''
    This class stores information of a cell in an image
    '''
    def __init__(self, ID, centroid, area, shape, div):
        self.id = ID
        self.c = centroid
        self.prev_c = None
        self.displacement = 0
        self.area = area
        self.shape = shape
        self.div = div
        
    def set_id(self, ID):
        self.id = ID
        
    def set_centroid(self, centroid):
        self.c = centroid
    
    def set_prev_centroid(self, prev_c):
        self.prev_c = prev_c
        
    def get_id(self):
        return self.id
    
    def get_centroid(self):
        return self.c
    
    def get_prev_centroid(self):
        return self.prev_c
    
    def get_area(self):
        return self.area
    
    def get_shape(self):
        return self.shape
    
    def get_div(self):
        return self.div
    
    def compute_displacement(self, D=30):
        if self.prev_c != None:
            v0 = self.c
            v1 = self.prev_c
            dist = np.sqrt((v0[0] - v1[0])**2 + (v0[1] - v1[1])**2)
            if dist > D:
                self.displacement = 0
            else:
                self.displacement = dist
            
        return self.displacement
    

In [92]:
def find_centroid(cnt):
    '''
    This function finds the centroid positions of a given contour
    '''
    m = cv2.moments(cnt)
    if m['m00'] != 0:
        cx = int(m['m10']/m['m00'])
        cy = int(m['m01']/m['m00'])
    else:
        cx, cy = 0, 0
    
    return (cx, cy)

def find_shape(cnt):
    '''
    This function finds the shape of a given contour using elliptic_fourier_descriptors
    '''
    contours = []
    for i in range(len(cnt)):
        contours.append(cnt[i][0])
    shape = elliptic_fourier_descriptors(contours, order=8, normalize=False)

    return shape

def calculate_distance(c0, c1, D=30):
    '''
    This function calculates the Euclidean distance between two given centroid positions,
    with a threshold D. 
    Output distance divided by D if distance is larger than D, otherwise output 1.
    '''
    dist = np.sqrt((c0[0] - c1[0])**2 + (c0[1] - c1[1])**2)
    if dist < D:
        return dist / D
    else:
        return 1

def shape_difference(s0, s1, order=8):
    '''
    This function calculates the elliptic_fourier_descriptors distance between two given shape
    '''
    if type(s0) is not int and type(s1) is not int:
        max_a, max_b, max_c, max_d = 0, 0, 0, 0
        dis = 0
        for o in range(order):
            dis += ((s0[o][0]-s1[o][0])**2+\
                    (s0[o][1]-s1[o][1])**2+\
                    (s0[o][2]-s1[o][2])**2+\
                    (s0[o][3]-s1[o][3])**2)
            max_a = max(max_a, (s0[o][0]-s1[o][0])**2)
            max_b = max(max_b, (s0[o][1]-s1[o][1])**2)
            max_c = max(max_c, (s0[o][2]-s1[o][2])**2)
            max_d = max(max_d, (s0[o][3]-s1[o][3])**2)
        dis /= (order * (max_a + max_b + max_c + max_d))
    else:
        dis = 1
    return dis

def identify_division(cnt):
    '''
    This function identifies whether a cell is in the process of dividing
    Return True for dividing, False for not dividing
    '''
    if len(cnt) >= 5:
        (x, y), (ma, MA), angle = cv2.fitEllipse(cnt)
        
        if ma != 0 and MA / ma > 2:
            return True
        else:
            return False
    else:
        return False

In [93]:
def find_match(cells_a, cells_b, a1, a2):
    '''
    This function finds the most similar match of a cell in two successive frames.
    '''
    match_list = []
    for c_a in cells_a:
        dist_list = [] # (distance, ID)
        for c_b in cells_b:
            if c_b.get_id() == -1 or c_a.get_id() == -1:
                cent_dist = 1
                shape_diff = 1
            else:
                cent_dist = calculate_distance(c_a.get_centroid(), c_b.get_centroid(), D=30)
                shape_diff = shape_difference(c_a.get_shape(), c_b.get_shape(), order=8)
            
            d = 1
            if c_a.get_div():
                d = 0
            dist = a1 * cent_dist + a2 * shape_diff * d
            dist_list.append((dist, c_a.get_id(), c_b.get_id()))
        dist_list.sort(key=lambda x:x[0]) # sort the list of tuples by distance
        match = (dist_list[0][1], dist_list[0][2]) # the most matching pair id
        match_list.append(match)
    return match_list

def find_cell(ID, cells):
    '''
    Find the cell with a given id in a list of cells
    Return False if not found
    '''
    for c in cells:
        if c.get_id() == ID:
            return c
    
    return False

def find_max_id(cell_list):
    '''
    This function finds the maximum id among all cells
    '''
    max_id = -1
    for cells in cell_list:
        for c in cells:
            if c.get_id() > max_id:
                max_id = c.get_id()
    return max_id 

def find_prev_max_id(cells):
    '''
    This function finds the maximum id within a list of cells
    '''
    max_id = -1
    for c in cells:
        if c.get_id() > max_id:
            max_id = c.get_id()
    return max_id 

def find_num_division(cells):
    '''
    This function finds the number of cells that are in the process of dividing
    '''
    num = 0
    for c in cells:
        if c.get_div():
            num += 1
    return num

def find_cell_count(cells):
    '''
    This function finds the number of cells in a given list of cell objects
    '''
    count = 0
    for c in cells:
        if c.get_id() != -1:
            count += 1
    return count
    

### Task 1.1 Draw Contours

In [94]:
cell_list = []

# Construct cell objects for each image
contour_list = []
for i in range(len(bmarks)):
    contours, hierarchy = cv2.findContours(bmarks[i], cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contour_list.append(contours)
    
    cells = []
    new_id = find_max_id(cell_list) + 1
    print ("Constructing cells of image: ", i)
    for i in range(len(contours)):
        cent = find_centroid(contours[i])
        area = round(cv2.contourArea(contours[i]), 2)
        shape = find_shape(contours[i])
        div = identify_division(contours[i])
        
        # ignore noises
        if area < 20:
            c = CELL(-1, cent, area, shape, False)
            cells.append(c)
        else:
            c = CELL(new_id, cent, area, shape, div)
            cells.append(c)
            new_id += 1
    
    cell_list.append(cells)
    clear_output(wait=True)
print ("Complete!")


Complete!


In [95]:
# Find matches and change ids accordingly for each pair of successive frames
for i in range(len(cell_list) - 1):
    print ("Finding matches of image: ", i)
    cells_a = cell_list[i]
    cells_b = cell_list[i + 1]
    
    matches = find_match(cells_a, cells_b, 0.5, 0.5)
    new_id = find_prev_max_id(cell_list[i]) + 1
    for j in range(len(matches)):

        a = find_cell(matches[j][0], cells_a) # find the cell in 1st image
        b = find_cell(matches[j][1], cells_b) # find the cell in 2nd image
        if a and b:
            b.set_id(matches[j][0]) # set the id in 2nd image to be the same as 1st
            b.set_prev_centroid(a.get_centroid())
    
    for cell in cells_b:
        if cell.get_id() != -1 and cell.get_prev_centroid() == None:
            cell.set_id(new_id)
            new_id += 1
    
    clear_output(wait=True)
print ("Complete!")


Complete!


In [96]:
# Generate random colors
max_id = find_max_id(cell_list) + 10
colors = [np.random.randint(0, 255, size=max_id),\
          np.random.randint(0, 255, size=max_id),\
          np.random.randint(0, 255, size=max_id)]
font = cv2.FONT_HERSHEY_SIMPLEX 

# Draw colors and add text on images
results = []
for i in range(len(imgs_normalized)):
    print ("Drawing colors on image: ", i)
    img_color = imgs_normalized[i].copy()
    img_color = cv2.cvtColor(img_color, cv2.COLOR_GRAY2RGB)
    
    j = 0
    for c in cell_list[i]:
        ID = c.get_id()
        if ID != -1:
            color = (int(colors[0][ID]), int(colors[1][ID]), int(colors[2][ID]))
            img_color = cv2.drawContours(img_color, contour_list[i], j, color, -1)
            img_color = cv2.putText(img_color, str(ID), c.get_centroid(), font, 0.5, (255, 255, 255), 1)
            if c.get_div():
                img_color = cv2.drawContours(img_color, contour_list[i], j, (255,0,0), 2)
        j += 1
    
    results.append(img_color)
    clear_output(wait=True)

os.chdir(PATH_TO_RESULTS)
print("Saving images as gif...")
imageio.mimsave('color_01_final.gif', results, duration=0.1)
clear_output(wait=True)
print ("Complete!")


Complete!


### Task 1.2 Show Trajectories

In [97]:
# Show trajectories of cells
path = []

for i in range(len(cell_list)):
    img_path = imgs_normalized[i].copy()
    img_path = cv2.cvtColor(img_path, cv2.COLOR_GRAY2RGB)
    
    print ("Drawing path on image: ", i)
    # Draw lines for the current frame and all previous frames
    for j in reversed(range(i)):
        for c in cell_list[j]:
            ID = c.get_id()
            prev_cent = c.get_prev_centroid()
            cent = c.get_centroid()
            color = (int(colors[0][ID]), int(colors[1][ID]), int(colors[2][ID]))
            disp = c.compute_displacement()
            if prev_cent != None and disp != 0 and (c.get_area() > 50):
                img_path = cv2.line(img_path, prev_cent, cent, color, 1)
        
    path.append(img_path)
    clear_output(wait=True)
    
os.chdir(PATH_TO_RESULTS)
print("Saving images as gif...")
imageio.mimsave('path_01_final.gif', path, duration=0.1)
clear_output(wait=True)
print ("Complete!")    


Complete!


### Task 2 Analyze Cell Motion

In [98]:
for i in range(len(cell_list)):
    cell_count = find_cell_count(cell_list[i])
    #print ("Number of cells in image " + str(i) + ": " + str(cell_count))
    
    area_sum = 0
    disp_sum = 0
    for c in cell_list[i]:
        if c.get_id() != -1:
            area_sum += c.get_area()
            #print(c.get_area())
            disp_sum += c.compute_displacement()
    avg_area = round((area_sum / cell_count), 2)
    
    #print("Total displacement: " + str(disp_sum))
    #print("Total area: " + str(area_sum))

    print("Average cell size in image " + str(i) + ": " + str(avg_area) + " pixels")

    avg_disp = round((disp_sum / cell_count), 2)
    #print("Average displacement in image " + str(i) + ": " + str(avg_disp) + " pixels")
    
    num_division = find_num_division(cell_list[i])
    #print("Number of divisions in image " + str(i) + ": " + str(num_division))

Average cell size in image 0: 344.72 pixels
Average cell size in image 1: 308.32 pixels
Average cell size in image 2: 277.52 pixels
Average cell size in image 3: 274.36 pixels
Average cell size in image 4: 247.95 pixels
Average cell size in image 5: 296.41 pixels
Average cell size in image 6: 249.69 pixels
Average cell size in image 7: 304.53 pixels
Average cell size in image 8: 289.95 pixels
Average cell size in image 9: 303.09 pixels
Average cell size in image 10: 272.82 pixels
Average cell size in image 11: 305.04 pixels
Average cell size in image 12: 291.89 pixels
Average cell size in image 13: 281.94 pixels
Average cell size in image 14: 255.47 pixels
Average cell size in image 15: 275.39 pixels
Average cell size in image 16: 309.45 pixels
Average cell size in image 17: 248.25 pixels
Average cell size in image 18: 234.46 pixels
Average cell size in image 19: 278.61 pixels
Average cell size in image 20: 290.21 pixels
Average cell size in image 21: 259.98 pixels
Average cell size in